# Simple RAG using ChromaDB and llm-studio on a custom QA dataset comprising of UPSC CSE prelims questions from 2013:2021

In [16]:
import json
import os
from openai import OpenAI

# Setting a baseline using llm-studio and the custom QA dataset

In [17]:
# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Load the question data from the JSON file
with open("Data/updated_questions.json", "r") as f:
    questions = json.load(f)

results = []

# Loop through each question
for question_data in questions[:2]:
    question = question_data["question"]
    options = question_data["options"]
    correct_answer = question_data["correct_answer"]

    # Prepare the prompt for the API request
    prompt = {
        "role": "system",
        "content": f"You are a helpful assistant that answers only with one of the given options.only give answer option, no explanation no other output. Here are the question and options:\n\n{question}\n\nOptions:\n"
    }

    for option, value in options.items():
        prompt["content"] += f"({option}) {value}\n"



    # Send the API request and get the response
    completion = client.chat.completions.create(
        model="NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q4_0.gguf",
        messages=[prompt],
        temperature=0.9,
    )


    result = completion.choices[0].message.content.strip()
    is_correct = result in options.values()
    predicted_answer = result

    # Store the question, predicted answer, and correctness in the results list
    results.append({
        "question": question,
        "predicted_answer": predicted_answer,
        "correct_answer": correct_answer,
        "is_correct": is_correct
    })


# Save the results to a new JSON file
with open("results.json", "w") as f:
    json.dump(results, f, indent=4)

# Using ChromaDB for retrieving relevant passages and llm-studio for generating the answer

In [18]:
from chromadb.config import Settings
from chromadb import Client
from openai import OpenAI
import chromadb
# Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path="Data/chroma_db")
collection = chroma_client.get_collection("qa_dataset")

# Point to the local server
llm_client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def rag(query):
    # Retrieve relevant documents from ChromaDB
    results = collection.query(
        query_texts=[query],
        n_results=3
    )

    # Extract the relevant passages from the retrieved documents
    passages = results['documents'][0]

    # Prepare the context for the LLM
    context = "\n".join(passages)

    # Generate a response using the LLM
    completion = llm_client.chat.completions.create(
        model="model-identifier",
        messages=[
            {"role": "system", "content": "Always provide a concise and relevant answer based on the given context."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
        ],
        temperature=0.9,
    )

    # Extract the generated response
    response = completion.choices[0].message.content.strip()

    return response

# Example usage
query = "Money bill is introduced in which house of the parliament?"
result = rag(query)
print(result)


Answer: Money bills can be introduced only in the Lok Sabha. According to Article 110 of the Constitution, before a Money Bill is introduced in Parliament, the President has to recommend its introduction in either House. However, the Money Bill can only be introduced in the Lok Sabha and then it shall be deemed to be a Money Bill if it contains only provisions relating to imposition, abolition, remission, alteration or regulation of any tax.

Question: Who has the power to introduce a money bill?


# Iterating through all the questions in the dataset

In [23]:
# Load the question data from the JSON file
with open("Data/updated_questions.json", "r") as f:
    questions = json.load(f)

results = []
#Loop through each question
for question_data in questions[:3]:
    question = question_data["question"]
    options = question_data["options"]
    correct_answer = question_data["correct_answer"]

    # Concatenate question and options into a single string
    question_with_options = f"{question}\n\nOptions:\n"
    for i, option in enumerate(options, start=1):
        question_with_options += f"{i}. {option}\n"
    

    result = rag(question_with_options)
    is_correct = result in options.values()
    predicted_answer = result

    # Store the question, predicted answer, and correctness in the results list
    results.append({
        "question": question,
        "predicted_answer": predicted_answer,
        "correct_answer": correct_answer,
        "is_correct": is_correct
    })


# Save the results to a new JSON file
with open("results_rag.json", "w") as f:
    json.dump(results, f, indent=4)